In [1]:
# Importação das bibliotecas
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

In [2]:
dataset = pd.read_csv("soybean.csv")
dataset

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,april,?,?,?,?,?,upper-areas,?,?,?,...,?,?,?,?,?,?,?,?,?,2-4-d-injury
679,april,lt-normal,?,lt-norm,?,diff-lst-year,scattered,?,?,?,...,?,dna,?,?,?,?,?,?,rotted,herbicide-injury
680,june,lt-normal,?,lt-norm,?,diff-lst-year,scattered,?,?,?,...,?,dna,?,?,?,?,?,?,rotted,herbicide-injury
681,april,lt-normal,?,lt-norm,?,same-lst-yr,whole-field,?,?,?,...,?,dna,?,?,?,?,?,?,rotted,herbicide-injury


In [3]:
#separação dos variáveis, ignoro primeira pois não tem valor semântico
X = dataset.iloc[:,0:35].values
y = dataset.iloc[:, 35].values
#temos um arry e não mais um data frame
X[0:2]

array([['october', 'normal', 'gt-norm', 'norm', 'yes', 'same-lst-yr',
        'low-areas', 'pot-severe', 'none', '90-100', 'abnorm', 'abnorm',
        'absent', 'dna', 'dna', 'absent', 'absent', 'absent', 'abnorm',
        'no', 'above-sec-nde', 'brown', 'present', 'firm-and-dry',
        'absent', 'none', 'absent', 'norm', 'dna', 'norm', 'absent',
        'absent', 'norm', 'absent', 'norm'],
       ['august', 'normal', 'gt-norm', 'norm', 'yes', 'same-lst-two-yrs',
        'scattered', 'severe', 'fungicide', '80-89', 'abnorm', 'abnorm',
        'absent', 'dna', 'dna', 'absent', 'absent', 'absent', 'abnorm',
        'yes', 'above-sec-nde', 'brown', 'present', 'firm-and-dry',
        'absent', 'none', 'absent', 'norm', 'dna', 'norm', 'absent',
        'absent', 'norm', 'absent', 'norm']], dtype=object)

In [4]:
labelencoder = LabelEncoder()

for x in range(35):
    X[:, x] = labelencoder.fit_transform(X[:, x])

In [5]:
#one hot encoder coluna credit_history
#deve adicionar 5 colunas
onehotencoder = make_column_transformer((OneHotEncoder(categories='auto', sparse_output=False), [1]), remainder="passthrough")
X = onehotencoder.fit_transform(X)
X[0:2]

array([[0.0, 0.0, 1.0, 6, 1, 3, 2, 4, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 1, 3, 1, 4, 4, 2, 1, 1, 2, 1, 2],
       [0.0, 0.0, 1.0, 2, 1, 3, 2, 3, 2, 3, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 2, 1, 1, 2, 2, 1, 3, 1, 4, 4, 2, 1, 1, 2, 1, 2]], dtype=object)

In [6]:
#Excluimos a variável para evitar a dummy variable trap
X = X[:,1:]
X = X.astype('float')
X

array([[0., 1., 6., ..., 2., 1., 2.],
       [0., 1., 2., ..., 2., 1., 2.],
       [0., 1., 3., ..., 2., 1., 2.],
       ...,
       [1., 0., 4., ..., 0., 0., 3.],
       [1., 0., 1., ..., 0., 0., 3.],
       [1., 0., 4., ..., 0., 0., 3.]])

In [7]:
#Laber encoder com a classe
labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)
classe_dummy = np_utils.to_categorical(y)
classe_dummy

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
#separação em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, classe_dummy, test_size = 0.2, random_state = 0)
print(len(X_train),len(X_test),len(y_train),len(y_test))

546 137 546 137


In [9]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
X_test

array([[0., 1., 4., ..., 2., 1., 2.],
       [1., 0., 2., ..., 0., 0., 3.],
       [1., 0., 5., ..., 0., 0., 3.],
       ...,
       [0., 1., 6., ..., 2., 1., 2.],
       [1., 0., 4., ..., 2., 1., 3.],
       [0., 1., 6., ..., 2., 1., 2.]])

In [11]:
classifier = Sequential()
classifier.add(Dense(units = 18, input_dim = 36))
classifier.add(Dense(units = 38))
classifier.add(Dense(units = 19, activation = 'softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                666       
                                                                 
 dense_1 (Dense)             (None, 38)                722       
                                                                 
 dense_2 (Dense)             (None, 19)                741       
                                                                 
Total params: 2,129
Trainable params: 2,129
Non-trainable params: 0
_________________________________________________________________


In [12]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
18/18 [==============================] - 1s 10ms/step - loss: 3.9239 - accuracy: 0.1099 - val_loss: 2.9098 - val_accuracy: 0.2044
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: 2.6431 - accuracy: 0.2509 - val_loss: 2.4585 - val_accuracy: 0.2701
Epoch 3/100
18/18 [==============================] - 0s 3ms/step - loss: 2.3268 - accuracy: 0.2985 - val_loss: 2.2526 - val_accuracy: 0.3358
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: 2.1457 - accuracy: 0.3443 - val_loss: 2.1116 - val_accuracy: 0.3650
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: 2.0058 - accuracy: 0.3938 - val_loss: 1.9961 - val_accuracy: 0.4015
Epoch 6/100
18/18 [==============================] - 0s 3ms/step - loss: 1.9274 - accuracy: 0.4194 - val_loss: 1.8958 - val_accuracy: 0.4307
Epoch 7/100
18/18 [==============================] - 0s 3ms/step - loss: 1.7885 - accuracy: 0.4670 - val_loss: 1.7956 - val_accuracy: 0.4599
Epoch 8/100


Epoch 59/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3659 - accuracy: 0.9084 - val_loss: 0.4198 - val_accuracy: 0.8905
Epoch 60/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3591 - accuracy: 0.9011 - val_loss: 0.4264 - val_accuracy: 0.8832
Epoch 61/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3483 - accuracy: 0.8974 - val_loss: 0.4073 - val_accuracy: 0.8905
Epoch 62/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3439 - accuracy: 0.8993 - val_loss: 0.4058 - val_accuracy: 0.8832
Epoch 63/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3373 - accuracy: 0.8993 - val_loss: 0.4038 - val_accuracy: 0.8832
Epoch 64/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3491 - accuracy: 0.8956 - val_loss: 0.3966 - val_accuracy: 0.8832
Epoch 65/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3407 - accuracy: 0.8883 - val_loss: 0.4055 - val_accuracy: 0.8832
Epoch 

In [13]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred[0:2]

5/5 [==============================] - 0s 750us/step


array([[False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False]])

In [14]:
y_teste_matrix = [np.argmax(t) for t in y_test]
y_previsao_matrix = [np.argmax(t) for t in y_pred]

In [15]:
confusao = confusion_matrix(y_teste_matrix, y_previsao_matrix)
confusao

array([[ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 5, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0],
       [ 0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 1,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 2,  1,  0,  0,  0, 14,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  